# Transfer Line Example

Transfer Lineの例です。  
授業で使う資料が以下のリンクにあるので参考にしてください。（特にLecture 6のところなど）  
**参考**  
[DES Course](https://www.control.eng.osaka-cu.ac.jp/teaching/des-course)   
[Lecture6](https://www.control.eng.osaka-cu.ac.jp/teaching/des-course/Lecture7.pptx)

## ざっくり説明
```
       ┌───────────┐      ┌───────────┐      ┌───────────┐     ┌──────────┐     ┌───────────┐
       │           │      │           │      │           │     │          │     │           │
──────►│ Machine 1 ├─────►│  Buffer 1 ├─────►│ Machine 2 ├────►│ Buffer 2 ├────►│ Test Unit ├─────►
       │           │      │           │      │           │     │          │     │           │
       └───────────┘      └───────────┘      └───────────┘     └──────────┘     └─────┬─────┘
                                ▲                                                     │
                                │                                                     │
                                │                                                     │
                                └─────────────────────────────────────────────────────┘
```

Machine 1, 2 と その間に Buffer 1, 2があり、最後にTest Unitがあります。

Machine 1に何かアイテムを入れると、Machineは加工を行い、終了するとBuffer 1にアイテムを吐き出します。   
Machine 2も同様にアイテムを入れると加工が始まり、終了するとBuffer 2にアイテムを吐き出します。   
Machine 1や2にアイテムを入れることは制御可能ですが、終了してBufferに吐き出すのは不可制御（制御できない）となります。  
Test Unitはアイテムのテストを行う機械であり、アイテムを入れると問題なければ次へ、問題があればBuffer １に戻します。  

制御の要求としてはBuffer1,2で保持できるアイテムの数は１個と制限されているので、Bufferがオーバーフロー(アイテムが２個以上になる）、またはアンダーフロー（アイテムがないのにMachineを動かそうとする）にならないようにする必要があります。

従って、Bufferが２個以上に増える可能性のある状況であったり、０個にも関わらず次のMachineを動かすといったような状況にならないように制御する必要があります。

## 普通のバージョン

### Initialize

In [ ]:
%pip install pitct

In [ ]:
import pitct

pitct.init('TRANLINE', overwrite=True)

### Step 1

Machine,Test Unit,Buffer それぞれの制御仕様を定義します

In [ ]:
# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,1,1), # transition triples (exit state, event, enter state)
         (1,2,0)] # Odd number event: Controllable, Even number event: Uncontrollable
pitct.create('M1', Q, delta, Qm) # create automaton

m1 = pitct.AutomatonDisplay('M1')
m1.render()

In [ ]:
# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,3,1), # transition triples (exit state, event, enter state)
         (1,4,0)  # Odd number event: Controllable, Even number event: Uncontrollable
         ]
pitct.create('M2', Q, delta, Qm) # create automaton

m2 = pitct.AutomatonDisplay('M2')
m2.render()

In [ ]:
# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,5,1), # transition triples (exit state, event, enter state)
         (1,6,0),   # Odd number event: Controllable, Even number event: Uncontrollable
         (1,8,0)
         ]
pitct.create('TU', Q, delta, Qm) # create automaton

tu = pitct.AutomatonDisplay('TU')
tu.render()

In [ ]:
# Creat specification (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,2,1), # transition triples (exit state, event, enter state)
         (0,8,1),  # Odd number event: Controllable, Even number event: Uncontrollable
         (1,3,0)
         ]
pitct.create('B1', Q, delta, Qm) # create automaton

pitct.AutomatonDisplay('B1')

In [ ]:
# Creat specification (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,4,1), # transition triples (exit state, event, enter state)
         (1,5,0)  # Odd number event: Controllable, Even number event: Uncontrollable
         ]
pitct.create('B2', Q, delta, Qm) # create automaton

pitct.AutomatonDisplay('B2')

### Step 2

制御仕様を同期合成します  
（Machine 1,2,Test Unitで起こり得る全パターンの遷移を考えたオートマトンの作成）

In [ ]:
pitct.sync('PLANT','M1','M2','TU')

pitct.AutomatonDisplay('PLANT')

### Step 3

制御要求を同期合成します

In [ ]:
pitct.allevents('ALL', 'PLANT')
pitct.sync('SPEC', 'B1', 'B2', 'ALL')

pitct.AutomatonDisplay('SPEC')

### Step 4

制御要求を満たすような制御を計算します。

In [ ]:
pitct.supcon('SUP', 'PLANT', 'SPEC')

sup = pitct.AutomatonDisplay('SUP')
sup.render()

In [ ]:
pitct.condat('SUPDAT', 'PLANT', 'SUP')
supdat = pitct.printdat('SUPDAT', 'SUPDAT')

In [ ]:
supdat

In [ ]:
supdat.control_data

## EVENTに名前をつけたバージョン

実行前に上のメニューバーのところから   
Kernel -> Restart Kernel  
を押してください。  

In [ ]:
import pitct

pitct.init('TRANLINE_WITH_NAME', overwrite=True)

In [ ]:
# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,'M1_in',1,'c'), # transition (exit state, event, enter state, 'c': controllable or 'u': uncontrollable)
         (1,'M1_out',0,'u')]
pitct.create('M1', Q, delta, Qm) # create automaton

m1 = pitct.AutomatonDisplay('M1')
m1.render()

In [ ]:
# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,'M2_in',1,'c'), # transition (exit state, event, enter state, 'c': controllable or 'u': uncontrollable)
         (1,'M2_out',0,'u')
         ]
pitct.create('M2', Q, delta, Qm) # create automaton

m2 = pitct.AutomatonDisplay('M2')
m2.render()

In [ ]:
# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,'TU_in',1, 'c'), # transition (exit state, event, enter state, 'c': controllable or 'u': uncontrollable)
         (1,'TU_ok',0, 'u'),
         (1,'TU_ng',0, 'u')
         ]
pitct.create('TU', Q, delta, Qm) # create automaton

tu = pitct.AutomatonDisplay('TU')
tu.render()

In [ ]:
# Creat specification (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,'M1_out',1, 'u'), # transition (exit state, event, enter state, 'c': controllable or 'u': uncontrollable)
         (0,'TU_ng',1, 'u'),
         (1,'M2_in',0, 'c')
         ]
pitct.create('B1', Q, delta, Qm) # create automaton

pitct.AutomatonDisplay('B1')

In [ ]:
# Creat specification (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [(0,'M2_out',1, 'u'), # transition (exit state, event, enter state, 'c': controllable or 'u': uncontrollable)
         (1,'TU_in',0, 'c')
         ]
pitct.create('B2', Q, delta, Qm) # create automaton

pitct.AutomatonDisplay('B2')

In [ ]:
pitct.sync('PLANT','M1','M2','TU')

pitct.AutomatonDisplay('PLANT')

In [ ]:
pitct.allevents('ALL', 'PLANT')
pitct.sync('SPEC', 'B1', 'B2', 'ALL')

pitct.AutomatonDisplay('SPEC')

In [ ]:
pitct.supcon('SUP', 'PLANT', 'SPEC')

sup = pitct.AutomatonDisplay('SUP')
sup.render()

In [ ]:
pitct.condat('DAT', 'PLANT', 'SUP')
dat = pitct.printdat('DAT', 'DAT')
dat.control_data[0]

In [ ]:
des = pitct.des_info('SUP')

des[0]['marked']

In [ ]:
## 今回はどうでもいいですが、状態に名前をつけることもできます。

# Creat plant (components)
Q = 2 # number of states
       # the initial state q0 is always labeled "0"
Qm = [0] # marker state set
delta = [('a','M1_in','b','c'), # transition (exit state, event, enter state, 'c': controllable or 'u': uncontrollable)
         ('b','M1_out','a','u')]
pitct.create('M1', Q, delta, Qm) # create automaton

m1 = pitct.AutomatonDisplay('M1')
m1.render()